In [2]:
# importing required modules 
import PyPDF2 
  
def extract_raw_text(file):
    
    pdfFileObj = open(file, "rb") 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    
    raw_text = ""
    for n in range(pdfReader.numPages):
        raw_text += pdfReader.getPage(n).extractText()
        
    ##raw_text = raw_text.split("\n") 
    
    pdfFileObj.close() 
    return raw_text

def extract_sections(raw_text):
    pass
            

raw_text = extract_raw_text("Omnirad-184.pdf")
raw_text


'      '

In [3]:
import os
import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import unicodedata, codecs
from io import StringIO
import io
##765-12-8_Sigma-Aldrich.pdf

In [4]:
laparams = pdfminer.layout.LAParams()
setattr(laparams, 'all_texts', True)

def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=laparams)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    if text:
        return text
    
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            
            text = fake_file_handle.getvalue()
            yield text
    
            # close open handles
            converter.close()
            fake_file_handle.close()
            
def extract_text(pdf_path):
    for page in extract_text_from_pdf(pdf_path):
        print(page)
        print() 
        
raw_text = extract_text_from_pdf("Omnirad-184.pdf")
#raw_text        

In [41]:
import re

regex_checkpoint = [
    "^[^.\n]*?(SECTION|section|Section)?[^\S\r\n]?\d{1,2}[^\S\r\n]?[.:-]?[^\S\r\n]*?(\w+[\s/]+)*?",
    "^[^a-zA-Z\n]*?(SECTION|section|Section)?[^\S\r\n]?\d{1,2}[^\S\r\n]?[.:-]?[^\S\r\n]*?(\w+[\s/]+)*?"
]

regex1 = "^[^a-zA-Z\n]*?(SECTION)?[^\S\r\n]?(?<!\.)"
regex2 = "[^\S\r\n]?[.:-]?[^\S\r\n]*?(\w+[\s/]+)*?"

sections = {
    1: "(IDENTIFICATION|Identification|identification)",
    2: "(HAZARDS|Hazard\(s\)|INGREDIENTS)",
    3: "(COMPOSITION|INGREDIENTS|HAZARDS|Hazard\(s\))",
    4: "(First[\s-]aid)",
    5: "(Fire[\s-]*Fighting)",
    6: "(Accidental\sRelease)",
    7: "(Handling\sAnd\sStorage)",
    8: "(Exposure\s(Controls\s*)?[And|/]*\s*?Personal\s(Protection)*)",
    9: "(Physical\s(And|&)\sChemical\sProperties)",
    10: "(Stability\s(And|&)\sReactivity)",
    11: "(Toxicological\sInformation)",
    12: "(Ecological\sInformation)",
    13: "(Disposal\sConsiderations*)",
    14: "(Transport(ation)*\sInformation)",
    15: "(Regulatory\sInformation)",
    16: "(Other\sInformation)"
}
old_sections = [
    ("(SECTION|section|Section)", "1"),
    ("hazard", "2"),
    ("ingredients", "3"),
    ("aid", "4"),
    ("fire", "5"),
    ("accidental", "6"),
    ("storage", "7"),
    ("exposure", "8"),
    ("properties", "9"),
    ("stability", "10"),
    ("toxicological information", "11"),
    ("ecological information", "12"),
    ("disposal considerations", "13"),
    ("transport", "14"),
    ("regulatory information", "15"),
    ("other information", "16")
]

def extract_section_positions(input_text):
    section_pos_list = []
    for n in range(1, len(sections) + 1):
        try:
            pos = re.search(regex1 + str(n) + regex2 + f"{sections[n]}", input_text, flags=re.MULTILINE | re.IGNORECASE).start()
            section_pos_list.append(pos)
        except Exception as e:
            print(f"Could not find section {n} due to {e} check regex or section is not present in file.")
    print(f"Position list: {section_pos_list}")
    return section_pos_list

                
def find_section_headers(input_text, positions):
    headers = []
    for pos in positions:
        section_text = input_text[pos:]
        #print(input_text[pos:])
        headers.append(section_text[:section_text.index("\n")].rstrip())
    return headers

def convert_headers_to_dict(headers):
    return {i : headers[i] for i in range(len(headers))}

def load_pdf_sections():
    for filename in os.listdir("."):
        if filename.endswith(".pdf"):
            print()
            print(f"Testing file: {filename}")
            try:
                raw_text = extract_text_from_pdf(filename)
                #print(raw_text)
                sections_list = find_section_headers(raw_text, extract_section_positions(raw_text))
                for n in sections_list:
                    print(n)
            
            except Exception as e:
                print(e)
            print()


#glycol section 3 & 8
#quilon section 9 & 10
#horizon blue 6-16
#load_pdf_sections()
convert_headers_to_dict(find_section_headers(raw_text, extract_section_positions(raw_text)))

Position list: [248, 1095, 3032, 4695, 5729, 6628, 7154, 7651, 9809, 12360, 13106, 15701, 20119, 20482, 22710, 22947]


{0: '1. Identification',
 1: '2. Hazards Identification',
 2: '3. Composition/Information on Ingredients',
 3: '4. First-Aid Measures',
 4: '5. Fire-Fighting Measures',
 5: '6. Accidental Release Measures',
 6: '7. Handling and Storage',
 7: '8. Exposure Controls/Personal Protection',
 8: '9. Physical and Chemical Properties',
 9: '10. Stability and Reactivity',
 10: '11. Toxicological Information',
 11: '12. Ecological Information',
 12: '13. Disposal Considerations',
 13: '14. Transport Information',
 14: '15. Regulatory Information',
 15: '16. Other Information'}

In [38]:

debug_text = extract_text_from_pdf("SDS - Horizon Blue Clarifier.pdf")
print(debug_text)

 

SAFETY DATA SHEET 

1. Identification

Product identifier 

Horizon Blue Clarifier 
None. 
Other means of identification 

Recommended 
use 

ALL PROPER AND LEGAL PURPOSES 

Recommended 

restrictions 

None known. 

Manufacturer/Importer/Supplier/Distributor 

information 

Manufacturer 

Company name 
Address 

Telephone 
E-mail

Horizon Chemical Co., Inc.
2125 Energy Park Drive
St. Paul, MN 55108
651-917-3075 
Not available.

Emergency 

phone number 

800-535-5053

INFOTRACK 

2.Hazard(s) 

identification

Physical 

hazards 

Not classified. 

Health hazards 

Not classified. 

Environmental 

hazards 

Not classified. 

OSHA defined hazards 

Not classified. 

Label elements 

Hazard symbol 

Signal word 

None. 

None. 

Hazard statement 

The substance 

does not meet the criteria 

for classification. 

Precautionary 

statement 

Prevention 

Observe good industrial 

hygiene practices. 

Response 

Wash hands after handling. 

Storage 

Store away from incompatible 

mate

In [6]:
HeaderDict = { 0 : "", \
               1 : ["1. PRODUCT AND COMPANY IDENTIFICATION", "SECTION 1: Identification", "SECTION 1: PRODUCT AND COMPANY IDENTIFICATION"," SECTION 1: Identification of the substance/mixture and of the company/undertaking ", "SECTION 1: Identification of the Substance/Preparation and the Company/Undertaking"], \
               2 : ["2. HAZARDS IDENTIFICATION", "SECTION 2: Hazard(s) identification"," SECTION 2 : Hazard(s) Identification"," SECTION 2: HAZARDS IDENTIFICATION","SECTION 2: Hazards Identification","SECTION 2: Hazards identification"],\
               3 : ["3. COMPOSITION/INFORMATION ON INGREDIENTS", "SECTION 3: Composition/Information on ingredients", "SECTION 3: Composition/Information on Ingredients", "SECTION 3: COMPOSITION/INFORMATION ON INGREDIENTS","SECTION 3:Composition/information on ingredients","SECTION 3: COMPOSITION"], \
               4 : ["4. FIRST AID MEASURES", "SECTION 4: First-aid measures","SECTION 4: First-Aid Measures","SECTION 4: FIRST AID MEASURES", " SECTION 4: First aid measures"], \
               5 : ["5. FIRE-FIGHTING MEASURES", "SECTION 5: Fire-fighting measures","SECTION 5: Fire-Fighting Measures", "SECTION 5: FIREFIGHTING MEASURES ","SECTION 5:FIRE-FIGHTING MEASURES","SECTION 5: FIRE FIGHTING MEASURES"," SECTION 5: Firefighting measures"], \
               6 : ["6. ACCIDENTAL RELEASE MEASURES", "SECTION 6: Accidental release measures", "SECTION 6: Accidental Release Measures", "SECTION 6: ACCIDENTAL RELEASE MEASURES "], \ 
               7 : ["7. HANDLING AND STORAGE", "SECTION 7: Handling and storage", " SECTION 7: Handling and Storage","SECTION 7: HANDLING AND STORAGE"], \
               8 : ["8. EXPOSURE CONTROLS/PERSONAL PROTECTION", "SECTION 8: Exposure controls/personal protection" , " SECTION 8: Exposure Controls/Personal Protection","SECTION 8: EXPOSURE CONTROLS/PERSONAL PROTECTION "," SECTION 8: EXPOSURE CONTROLS AND PERSONAL PROECTION ", "SECTION 8: 8. Exposure Controls/personal protection"], \
               9 : ["9. PHYSICAL AND CHEMICAL PROPERTIES", "SECTION 9: Physical and chemical properties", " SECTION 9: Physical and Chemical Properties", "SECTION 9: PHYSICAL AND CHEMICAL PROPERTIES"], \
               10 : ["10. STABILITY AND REACTIVITY", "SECTION 10: Stability and reactivity", " SECTION 10: Stability and Reactivity","SECTION 10: STABILITY AND REACTIVITY"], \
               11 : ["11. TOXICOLOGICAL INFORMATION", "SECTION 11: Toxicological information", " SECTION 11: Toxicological Information", "SECTION 11: TOXICOLOGICAL INFORMATION"], \
               12 : ["12. ECOLOGICAL INFORMATION", "SECTION 12: Ecological information", " SECTION 12: Ecological Information", "SECTION 12: ECOLOGICAL INFORMATION"], \
               13 : ["13. DISPOSAL CONSIDERATIONS", "SECTION 13: Disposal considerations", " SECTION 13: Disposal Considerations","SECTION 13: DISPOSAL CONSIDERATIONS"], \
               14 : ["14. TRANSPORT INFORMATION", "SECTION 14: Transport information", " SECTION 14: Transport Information","SECTION 14: TRANSPORT INFORMATION"], \
               15 : ["15. REGULATORY INFORMATION", "SECTION 15: Regulatory information", "SECTION 15: Regulatory Information", "SECTION 15: REGULATORY INFORMATION"], \
               16 : ["16. OTHER INFORMATION", "SECTION 16: Other information", "SECTION 16: Other Information","SECTION 16: OTHER INFORMATION", " SECTION 16: Other information, including date of preparation or last revision"], \
             }

SyntaxError: unexpected character after line continuation character (<ipython-input-6-d4ac63953b2c>, line 7)